In [ ]:
import pandas as pd
import re  # regex
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

: 

In [ ]:
# previewing iniail values
df = pd.read_csv("imdb_master.csv", sep=",", encoding="ISO-8859-1")
df.head()

: 

In [ ]:
# checking initail values
print(df['type'].unique())
print(df['label'].unique())

: 

In [ ]:
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[3], axis=1)

df

: 

In [ ]:
nltk.download("stopwords")
stop_words = stopwords.words("english")
CLEANR = re.compile("<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
nltk.download("punkt")
ps = PorterStemmer()
nltk.download("wordnet")
nltk.download("omw-1.4")

wnl = WordNetLemmatizer()

def remove_stop_words(sentence):
    # Split the sentence into individual words
    words = sentence.split()
    # Use a list comprehension to remove stop words
    filtered_words = [word for word in words if word not in stop_words]
    # Join the filtered words back into a sentence
    return " ".join(filtered_words)


def remove_html_tags(sentence):
    return re.sub(CLEANR, "", sentence)


def remove_punctuation(sentence):
    return sentence.translate(str.maketrans("", "", string.punctuation))


def remove_none_words(sentence):
    sentence = re.sub(r"[0-9]+", "", sentence)
    return re.sub(r"\W+", " ", sentence)


def apply_stemmer(sentence):
    return ps.stem(sentence)


def apply_lemmatization(sentence):
    return wnl.lemmatize(sentence)


def get_words_tokens(sentence):
    words_tokens = word_tokenize(sentence)
    new_text = " ".join([wnl.lemmatize(ps.stem(word)) for word in words_tokens] )
    return new_text

: 

In [ ]:
# rows data clean
review_token = []
for index, row in df.iterrows():
    row["review"] = remove_html_tags(row["review"])
    row["review"] = remove_punctuation(row["review"])
    row["review"] = remove_stop_words(row["review"])
    row["review"] = remove_none_words(row["review"])
    row["review"] = row["review"].lower()
    row["review"] = apply_stemmer(row["review"])
    row["review"] = apply_lemmatization(row["review"])
    row["review"] = get_words_tokens(row["review"])

df.head()

: 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df["review"], df["label"], test_size=0.1)
tfidf = TfidfVectorizer(norm=None, smooth_idf=False)

# train_data = df.query('type == "train"')
# test_data = df.query('type == "test"')

# x_train = train_data["review_token"]
# y_train = train_data["label"]

# x_test = test_data["review_token"]
# y_test = test_data["label"]


x_train_victor = tfidf.fit_transform(x_train)
x_test_victor = tfidf.transform(x_test)

clf = MultinomialNB()
clf.fit(x_train_victor, y_train)

: 